In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

print(tf.__version__)
print(np.__version__)

2.4.0
1.19.5


# Model Building

In [5]:
# Define the model architecture
model = Sequential()
model.add(LSTM(100, activation='tanh', return_sequences=True, input_shape=(378, 80)))
model.add(Dropout(0.2))
model.add(LSTM(100, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='tanh'))

In [6]:
# Compile the model
adam = Adam(learning_rate=0.001)
model.compile(optimizer=adam, loss='mean_squared_error')

In [7]:
# Print the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 378, 100)          72400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 378, 100)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 303       
Total params: 153,103
Trainable params: 153,103
Non-trainable params: 0
_________________________________________________________________


# Model training

In [16]:
import pandas as pd

# Load the training and testing data
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

print(train_data.shape)
train_data.head()

(1938, 2)


,x,y
0,[0.00029355 0.00027799 0. 0. 0...,0.000262
1,[0.00027799 0. 0. 0. 0...,0.000000
2,[0. 0. 0. 0. 0...,0.000302
3,[0. 0. 0. 0. 0...,0.059955
4,[0. 0. 0. 0. 0...,0.000000


In [18]:

# Convert the dataframes to numpy arrays
x_train = train_data.drop('x', axis=1).values
y_train = train_data['y'].values
x_test = test_data.drop('x', axis=1).values
y_test = test_data['y'].values

# x_train.reshape(1,378,80)


ValueError: cannot reshape array of size 1938 into shape (1,378,80)

In [15]:
print(x_train.shape)
print(y_train.shape)

(1938, 1, 1)
(1938,)


In [14]:
# Reshape input to be 3D [samples, timesteps, features]
# x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))

# Fit the model
history = model.fit(
    x_train, y_train,
    epochs=10,  # adjust this value based on your requirements
    batch_size=32,  # adjust this value based on your requirements
    validation_split=0.1,  # 10% of the data will be used for validation
    shuffle=False
)

Epoch 1/10


ValueError: in user code:

    /Users/ethanwalker/opt/anaconda3/envs/DS/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /Users/ethanwalker/opt/anaconda3/envs/DS/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/ethanwalker/opt/anaconda3/envs/DS/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/ethanwalker/opt/anaconda3/envs/DS/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/ethanwalker/opt/anaconda3/envs/DS/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        retur